In [13]:
from utils.data_set import get_data_json, extract_data_set_info
from utils.matrix import fix_diagonal_weight
from itertools import product
from mip import CutType
from model.model import BSS
from typing import Any, Dict, Set, List, Tuple
import networkx as nx
import mip
import pandas as pd

In [14]:
try:
    data_set = get_data_json("../../data/ReggioEmilia/4ReggioEmilia30.json")

    (num_vertices, demands, vehicle_capacity, distance_matrix) = extract_data_set_info(
        data_set
    )

    fix_diagonal_weight(distance_matrix)

    V = set(range(num_vertices))
    A = {(i, j): distance_matrix[i, j] for i in V for j in V}
    m = 1
    Q = vehicle_capacity
    q = demands
    c = distance_matrix
    
    problem = BSS(
        V,
        A,
        m,
        Q,
        q,
        c
    )
except Exception as e:
    raise e

2024-08-26 15:22:49.761 | SUCCESS  | utils.data_set:get_data_json:23 - JSON file loaded correctly


In [15]:
problem.V, len(problem.A)

({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}, 196)

In [16]:
problem.Q, problem.q

(30, [0, -3, 1, 2, -5, 2, -10, -2, -2, 3, -1, -6, -9, 2])

In [17]:
df = pd.DataFrame(problem.c)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,3700,4000,3900,4900,3900,4800,3800,4000,2600,4600,3800,3000,3500
1,4300,0,1300,300,2400,1200,1300,2600,1600,1200,1000,600,1400,800
2,4500,900,0,900,2100,900,1000,1900,1300,2100,700,1400,2200,500
3,4500,300,1300,0,2200,1200,1000,2500,1600,1500,700,800,1600,800
4,5500,2200,2600,2000,0,3100,1200,3600,3200,2500,1800,2000,2600,2700
5,3700,800,900,1100,2900,0,1800,1500,500,2000,1500,1300,1700,400
6,5300,1400,1800,1100,2000,2300,0,2700,2300,2200,900,1800,2400,1900
7,3500,2200,1700,2400,3600,1500,2400,0,1400,3000,2400,2600,2700,1800
8,4300,800,1100,1000,3100,1000,2000,1500,0,1600,1700,1200,1300,600
9,3300,1600,2600,1800,2300,2500,2200,3000,2700,0,2100,1200,800,2100


In [18]:
problem.t = [problem.model.add_var(name=f"t_{j}", var_type=mip.CONTINUOUS) for j in V]

In [19]:
for j in V:
    problem.model += problem.t[j] >= max(0, q[j])
    problem.model += problem.t[j] <= min(Q, Q + q[j])

for i in V:
    for j in V:
        if j != 0 and i != j:
            # M = min(Q, Q + q[j])
            M = 10000
            problem.model += problem.t[j] >= problem.t[i] + q[j] - M * (1 - problem.x[i][j])

for i in V:
    for j in V:
        if i != 0 and i != j:
            # M = min(Q, Q - q[j])
            M = 10000
            problem.model += problem.t[i] >= problem.t[j] - q[j] - M * (1 - problem.x[i][j])

In [20]:

new_contraints = True
while new_contraints:
    problem.model.optimize(relax=True)

    print(
        "status: {} objective value : {}".format(
            problem.model.status, problem.model.objective_value
        )
    )

    G = nx.DiGraph()
    for i, j in A:
        G.add_edge(i, j, capacity=problem.x[i][j].x)

    new_contraints = False

    for u, v in [(i, j) for (i, j) in product(V, V) if i != j]:
        cut_value, (S, NS) = nx.minimum_cut(G, u, v)
        if cut_value <= 0.99 and 0 not in S and len(S) >= 1:
            problem.model += (
                mip.xsum(problem.x[i][j] for i, j in A if i in S and j in S)
                <= len(S) - 1
            )

            new_contraints = True
    if not new_contraints and problem.model.solver_name.lower() == "cbc":
        cp = problem.model.generate_cuts(
            [
                CutType.GOMORY,
                CutType.MIR,
                CutType.ZERO_HALF,
                CutType.KNAPSACK_COVER,
            ]
        )

        if cp.cuts:
            problem.model += cp
            new_contraints = True


Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 338 (-59) rows, 169 (-41) columns and 1248 (-212) elements
Clp1000I sum of infeasibilities 0.106973 - average 0.000316488, 127 fixed columns
Coin0506I Presolve 47 (-291) rows, 18 (-151) columns and 133 (-1115) elements
Clp0006I 0  Obj 17500.862 Primal inf 0.045375057 (9) Dual inf 3.9793382e+13 (15)
Clp0029I End of values pass after 18 iterations
Clp0014I Perturbing problem by 0.001% of 37.359704 - largest nonzero change 1.5894324e-05 ( 0.0013138967%) - largest zero change 2.7631883e-05
Clp0000I Optimal - objective value 17500.873
Clp0000I Optimal - objective value 17500.873
Coin0511I After Postsolve, objective 17500.873, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj 17500.873 Dual inf 18933.333 (36)
Clp0014I Perturbing problem by 0.001% of 0.97304428 - largest nonzero change 2.9646427e-05 ( 0.0014821731%) - largest zero change 2.849718e-05
Clp0000I Optimal - objective va

In [21]:
if problem.model.num_solutions:
    route = [(i, j) for i in V for j in V if problem.x[i][j].x >= 0.5]
    print(range(len(problem.t)))
    t_route = [problem.t[j].x for j in range(len(problem.t))]

    for i, j in route:
        print(f"{problem.model.var_by_name(f"x_{i}_{j}")} = {problem.x[i][j].x}")

range(0, 14)
x_0_9 = 1.0
x_1_11 = 0.999670588235293
x_2_10 = 0.9998352941176457
x_3_1 = 0.9996705882352936
x_4_6 = 0.9999999999999998
x_5_8 = 1.0
x_6_3 = 0.9996705882352936
x_7_5 = 0.9996705882352938
x_8_13 = 0.9996705882352932
x_9_7 = 0.9996705882352944
x_10_4 = 0.9998352941176457
x_11_12 = 1.0000000000000024
x_12_0 = 1.0
x_12_9 = 0.9999999999999998
x_13_2 = 0.9995058823529415


In [22]:
demands_after_solving = [problem.t[j].x for j in range(len(problem.t))]
q, demands_after_solving

([0, -3, 1, 2, -5, 2, -10, -2, -2, 3, -1, -6, -9, 2],
 [0.0,
  11.70588235295753,
  18.823529411751544,
  11.411764705897317,
  16.117647058836994,
  9.588235294116032,
  6.117647058834857,
  4.294117647054918,
  7.588235294113588,
  3.0000000000001137,
  19.470588235294578,
  9.00000000002592,
  0.0,
  12.88235294116587])